In [77]:
import edgar
import importlib
import pandas as pd

In [ ]:
edgar.get_docs('amzn', 8, ['10-Q', '10-K'])

In [149]:
importlib.reload(edgar)

AMZN_TAGS = {
    'amzn:fulfillmentexpense': 'Fulfillment',
    'amzn:technologyandinfrastructureexpense': 'Technology and Infrastructure',
    'amzn:technologyandcontentexpense': 'Technology and Content'
}
amzn_folders = ['2023Q3', '2023Q2', '2023Q1', '2022Q4', '2022Q3', '2022Q2', '2022Q1']
data = []

for folder in amzn_folders:
    folder_path = f"edgar_data/amzn/{folder}/"
    ops_context_ref = edgar.get_context_ref_by_name(f"{folder_path}MetaLinks.json", 'Consolidated Statements of Operations')
    print(folder, ops_context_ref)
    data.append(edgar.get_report_data_by_ref(f"{folder_path}ReportData.xml", ops_context_ref, AMZN_TAGS))
pd.pandas.set_option('display.max_columns', None)
df = pd.DataFrame(data=data, columns=[
    'Revenue',
    'COGS',
    'Fulfillment',
    'Technology and Infrastructure',
    'Technology and Content',
    'Sales and Marketing',
    'General Administrative',
    'Other Operating Expenses (Income) Net',
    'Operating Expenses',
    'Operating Income',
    'Interest Income',
    'Interest Expense',
    'Other Income (Expense) Net',
    'Total Non-Operating Income (Expense)',
    'Pretax Income',
    'Taxes',
    'Equity-method investment activity, net of tax',
    'Net income (loss)',
    'EPS',
    'Shares',
    'start_date',
    'end_date'
]).fillna(0)
df


2023Q3 c-10
2023Q2 c-10
2023Q1 id5a05ce5ce084612b9b02c88a7bcfc49_D20230101-20230331
2022Q4 i66a08f9a87424ace8d8c03c38fb30db9_D20220101-20221231
2022Q3 i6a7be016b1e14018ba8f205a6d0b0ff6_D20220701-20220930
2022Q2 if1c898010eef4246b9b88f9dfa8610f5_D20220401-20220630
2022Q1 iee090cd6862849ad8e3026c04d3a2165_D20220101-20220331


,Revenue,COGS,Fulfillment,Technology and Infrastructure,Technology and Content,Sales and Marketing,General Administrative,Other Operating Expenses (Income) Net,Operating Expenses,Operating Income,Interest Income,Interest Expense,Other Income (Expense) Net,Total Non-Operating Income (Expense),Pretax Income,Taxes,"Equity-method investment activity, net of tax",Net income (loss),EPS,Shares,start_date,end_date
0,143083,75022,22314,21203.0,0.0,10551,2561,-244,131895,11188,776,806,1031,1001,0.0,0.0,-4,9879,0.94,10558,2023-07-01,2023-09-30
1,134383,69373,21305,21931.0,0.0,10745,3202,-146,126702,7681,661,840,61,-118,0.0,0.0,-9,6750,0.65,10449,2023-04-01,2023-06-30
2,127358,67791,20905,0.0,20450.0,10172,3043,-223,122584,4774,611,823,-443,-655,0.0,0.0,1,3172,0.31,10347,2023-01-01,2023-03-31
3,513983,288831,84299,0.0,73213.0,42238,11891,-1263,501735,12248,989,2367,-16806,-18184,0.0,0.0,-3,-2722,-0.27,10189,2022-01-01,2022-12-31
4,127101,70268,20583,0.0,19485.0,11014,3061,-165,124576,2525,277,617,759,419,0.0,0.0,-3,2872,0.28,10331,2022-07-01,2022-09-30
5,121234,66424,20342,0.0,18072.0,10086,2903,-90,117917,3317,159,584,-5545,-5970,0.0,0.0,-12,-2028,-0.20,10175,2022-04-01,2022-06-30
6,116444,66499,20271,0.0,14842.0,8320,2594,-249,112775,3669,108,472,-8570,-8934,0.0,0.0,-1,-3844,-7.56,509,2022-01-01,2022-03-31


In [167]:
importlib.reload(edgar)

print(edgar.npv(df['Revenue'], 0.05))

df2 = df.drop(['start_date', 'end_date'], axis=1)
df2 = df2.transpose()
df2.columns = df['end_date'].dt.to_period('Q')
df2

879562.1211277585


end_date,2023Q3,2023Q2,2023Q1,2022Q4,2022Q3,2022Q2,2022Q1
Revenue,143083.00,134383.00,127358.00,513983.00,127101.00,121234.0,116444.00
COGS,75022.00,69373.00,67791.00,288831.00,70268.00,66424.0,66499.00
Fulfillment,22314.00,21305.00,20905.00,84299.00,20583.00,20342.0,20271.00
Technology and Infrastructure,21203.00,21931.00,0.00,0.00,0.00,0.0,0.00
Technology and Content,0.00,0.00,20450.00,73213.00,19485.00,18072.0,14842.00
Sales and Marketing,10551.00,10745.00,10172.00,42238.00,11014.00,10086.0,8320.00
General Administrative,2561.00,3202.00,3043.00,11891.00,3061.00,2903.0,2594.00
Other Operating Expenses (Income) Net,-244.00,-146.00,-223.00,-1263.00,-165.00,-90.0,-249.00
Operating Expenses,131895.00,126702.00,122584.00,501735.00,124576.00,117917.0,112775.00
Operating Income,11188.00,7681.00,4774.00,12248.00,2525.00,3317.0,3669.00
